<a href="https://colab.research.google.com/github/AycaCicek/Turkcell-ML-DogrusalOlmayanRegresyonModelleri/blob/main/YapaySinirA%C4%9Flar%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.preprocessing import scale,StandardScaler
from sklearn import model_selection
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import neighbors
from sklearn.svm import SVR

In [2]:
data=pd.read_csv("/content/Hitters.csv")
data.head(3)

,AtBat,Hits,HmRun,Runs,RBI,Walks,Years,CAtBat,CHits,CHmRun,CRuns,CRBI,CWalks,League,Division,PutOuts,Assists,Errors,Salary,NewLeague
0,293,66,1,30,29,14,1,293,66,1,30,29,14,A,E,446,33,20,NaN,A
1,315,81,7,24,38,39,14,3449,835,69,321,414,375,N,W,632,43,10,475.0,N
2,479,130,18,66,72,76,3,1624,457,63,224,266,263,A,W,880,82,14,480.0,A


In [7]:
data=data.dropna()
data_encoding=pd.get_dummies(data[["League","Division","NewLeague"]])

#BAGIMLI DEGISKEN
y=data["Salary"]

#BAGIMSIZ DEGISKENLER
X_=data.drop(["Salary","League","Division","NewLeague"],axis=1).astype("float64")
X=pd.concat([X_,data_encoding[["League_N","Division_W","NewLeague_N"]]],axis=1)

In [8]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25,random_state=42)

# MODEL VE TAHMİN

In [9]:
scaler=StandardScaler()

In [10]:
scaler.fit(X_train,y_train)
X_train_scaled=scaler.transform(X_train)

In [12]:
X_train_scaled[0:3]

array([[-0.52262793, -0.36609208,  0.02406249, -0.12266806, -0.34369699,
        -0.40963247, -1.10471461, -0.99114633, -0.945558  , -0.6928145 ,
        -0.91630243, -0.87918027, -0.85228393, -0.52007444, -0.39867767,
        -0.07254437,  1.02571114, -1.02571114,  1.06831847],
       [ 0.78285524,  0.86375586, -1.33031197,  0.52498644,  0.09159897,
         1.79077621,  0.37575327,  0.91009107,  0.68273257, -0.68101809,
         0.66499445,  0.12828482,  1.04608216, -0.24784721,  2.41907533,
         1.03899663,  1.02571114, -1.02571114,  1.06831847],
       [ 1.33733466,  1.04939328,  1.26557241,  0.60594325,  0.92261853,
         0.54706696, -0.25873296,  0.05654487, -0.04886217,  0.73455104,
         0.02890887,  0.15881406,  0.25221998,  0.03086163, -0.74910888,
        -0.39012751, -0.97493336,  0.97493336, -0.93605047]])

In [14]:
X_test_scaled=scaler.transform(X_test)

In [15]:
mlp_model=MLPRegressor().fit(X_train_scaled,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [18]:
mlp_model.predict(X_test_scaled)[0:5]

array([21.42201971, 42.28957879, 73.2326384 , 12.23038147, 23.20314099])

In [20]:
y_pred=mlp_model.predict(X_test_scaled)
np.sqrt(mean_squared_error(y_pred,y_test))

660.15970399997

Model Tuning

In [47]:
mlp_model.get_params().keys()

dict_keys(['activation', 'alpha', 'batch_size', 'beta_1', 'beta_2', 'early_stopping', 'epsilon', 'hidden_layer_sizes', 'learning_rate', 'learning_rate_init', 'max_fun', 'max_iter', 'momentum', 'n_iter_no_change', 'nesterovs_momentum', 'power_t', 'random_state', 'shuffle', 'solver', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [48]:
mlp_model.get_params()

{'activation': 'relu',
 'alpha': 0.0001,
 'batch_size': 'auto',
 'beta_1': 0.9,
 'beta_2': 0.999,
 'early_stopping': False,
 'epsilon': 1e-08,
 'hidden_layer_sizes': (100,),
 'learning_rate': 'constant',
 'learning_rate_init': 0.001,
 'max_fun': 15000,
 'max_iter': 200,
 'momentum': 0.9,
 'n_iter_no_change': 10,
 'nesterovs_momentum': True,
 'power_t': 0.5,
 'random_state': None,
 'shuffle': True,
 'solver': 'adam',
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': False,
 'warm_start': False}

In [56]:
mlp_params={"alpha":[0.1,0.01,0.02,0.001,0.0001],"hidden_layer_sizes":[(10,20),(5,5),(100,100)]}
#hidden layer gizli katman demektir parantez içine 2 değer yazdığında yukarıdaki gibi bu demektir ki,
#2 tane gizli katman oluştur.Değerler ise nöron sayılarıdır

In [65]:
mlp_cv_model=GridSearchCV(mlp_model,mlp_params,cv=10,verbose=2,n_jobs=-1).fit(X_train_scaled,y_train)

Fitting 10 folds for each of 15 candidates, totalling 150 fits


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [66]:
mlp_cv_model.best_params_

{'alpha': 0.0001, 'hidden_layer_sizes': (100, 100)}

In [67]:
mlp_tunned=MLPRegressor(alpha=0.0001,hidden_layer_sizes=(100,100)).fit(X_train_scaled,y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


In [68]:
y_pred=mlp_tunned.predict(X_test_scaled)

In [69]:
np.sqrt(mean_squared_error(y_pred,y_test))

361.0646470308828